In [4]:
import mindspore.nn as nn
#构建一个残差单元
class basic_res(nn.Cell):
    """
    需要设置的参数：
    input_channels, output_channels, stride
    """
    def __init__(self, input_channels, output_channels, stride = 1):
        super(basic_res, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = input_channels, out_channels = output_channels, kernel_size = 3, stride = stride, pad_mode="same")
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = output_channels, out_channels = output_channels, kernel_size = 3, stride = 1, pad_mode="same") #第二个卷积层的步长都为1，不需要人为设置
        self.downsample = nn.Conv2d(in_channels = input_channels, out_channels = output_channels, kernel_size = 1, stride = stride, pad_mode="same") #保证残差的输入shape与残差输出shape相同
    def construct(self, x):
        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.conv2(out)
        identity = self.downsample(x)
        
        out = out + identity
        out = self.relu(out)
        
        return out 

In [5]:
#堆叠残差单元构建成一个残差结构
def build_res(input_channels, output_channels,blocks, stride = 1):
    
    res_build = nn.SequentialCell()
    
    res_build.append(basic_res(input_channels, output_channels, stride = stride)) #第一个残差单元步长会改变，为2，具有下采样功能
    
    for _ in range(1, blocks):
        res_build.append(basic_res(output_channels, output_channels, stride = 1))#在一个残差结构里，除了第一个残差单元，后面步长均为1
    
    return res_build

In [6]:
#构建残差网络
from mindspore import nn
 
class Resnet(nn.Cell):
    
    def __init__(self, layer_dims, num_classes):
        super(Resnet, self).__init__()
        
        #输入层--对原始输入进行卷积池化等预处理
        self.stem = nn.SequentialCell([nn.Conv2d(3, 64,  7, 2, pad_mode='same'),  
                                nn.BatchNorm2d(64),      
                                nn.ReLU(),       
                                nn.MaxPool2d(3, 2, pad_mode='same')])
        #隐藏层---残差结构、卷积
        self.layer1 = build_res(64, 64, layer_dims[0])
        self.layer2 = build_res(64, 128, layer_dims[1], 2)
        self.layer3 = build_res(128, 256, layer_dims[2], 2)
        self.layer4 = build_res(256, 512, layer_dims[3], 2)
        
        #平均池化
        self.avgpool = nn.AvgPool2d(7, 1)
        
        #展开
        self.flatten = nn.Flatten()
 
        
        #全连接
        self.fc = nn.Dense(512, num_classes)
        
    def construct(self, x):
        #输入层
        out = self.stem(x)
        
        #隐藏层
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        #输出层
        out = self.avgpool(out)
        out = self.flatten(out)
        out = self.fc(out)
        
        return out      

In [20]:
from mindspore import context

# 使用昇腾
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")


#定义好每一个残差结构中残差单元数目
layer_dims = [2,2,2,2]

#建立测试数据
from mindspore import Tensor
from mindspore import numpy as np
x = Tensor(np.ones([1,3,224,224]))

#实例化网络
resnet18 = Resnet(layer_dims, 10)
#输入数据
x_resnet18 = resnet18(x)
x_resnet18

Tensor(shape=[1, 10], dtype=Float32, value=
[[-2.15314780e-10,  2.53866067e-10,  1.08013598e-10 ...  3.91784355e-10,  3.26666194e-10,  2.53374040e-11]])

In [ ]:
import numpy as np
from PIL import Image
import mindspore
import mindspore.ops as ops
from mindspore import Tensor
 
dic_ds_test = mnist_ds_test.create_dict_iterator(output_numpy = True) #创建迭代数据，返回字典类型，数据类型是数组
ds_test = next(dic_ds_test)  #取创建好的迭代数据
 
images_test = ds_test["image"]  
labels_test = ds_test["label"]
 
output = model.predict(Tensor(images_test))   #开始预测，返回一个每一类的预测分数
pred_labels = ops.Argmax(output_type=mindspore.int32)(output)  #返回预测分数中最大值的索引，即预测值
 
 
print("预测值 -- > ", pred_labels)  # 打印预测值
print("真实值 -- > ", labels_test)  # 打印真实值
 
 
batch_img = np.squeeze(images_test[0])
for i in range(1, len(labels_test)):
    batch_img = np.hstack((batch_img, np.squeeze(images_test[i])))  # 将一批图片水平拼接起来，方便下一步进行显示
Image.fromarray((batch_img*255).astype('uint8'), mode= "L")  # 显示真实值